# Poznański Horyzont Danych Meetup - 10.12.2024
# DSPy Introduction

Tutorial inspired by [dspy/examples/agents/multi_agent.ipynb](https://github.com/stanfordnlp/dspy/blob/main/examples/agents/multi_agent.ipynb)

If you've ever felt intimidated by DSPy, don't worry—it might look complex at first glance, but it's actually quite approachable. This tutorial will walk you through the process of building a  project, providing a clear, step-by-step approach to understanding and implementing DSPy concepts.

## Useful links and concepts before starting to rock DSPy... 🚀

- I strongly recommend reading the [DSPy Cheatsheet](https://dspy.ai/cheatsheet/) it will help you with quick start;

- Do you want know more about DSPY optimizers? Read [this guide](https://dspy.ai/learn/optimization/optimizers/?h=optimizers)!

- You find tons of tutorials in [dpsy/examples](https://github.com/stanfordnlp/dspy/blob/main/examples) folder;

- Remember the DSPy "scientific method":
  - **Define your task**: what is the input? What is the expected output?
  - **Define your pipeline**: do we need ChainOfThoughts? Do we need any external tool, like a retriever?
  - **Explore a few examples**: explore both input and output. It might be worth to annotate few examples for debugging purposes.
  - **Define your data**: define training and validation sets;
  - **Define your metric**: define an objective metric you will use to evaluate your programs. You don't have it? You can ask an LLM to evaluate for you!
  - **Collect preliminary "zero-shot" evaluations**: try out your program without optimization to get baseline results;
  - **Compile with a DSPy optimizer**: start with FewShot optimizers and follow with more complex ones, like MIPRO, COPROv2...
  - **Iterate**: iterate the steps above, until you are satisfief with the results! 

In [1]:
import os
import re

import dspy
import pandas as pd
from dotenv import load_dotenv
from dspy.evaluate import Evaluate
from dspy.datasets.hotpotqa import HotPotQA
from dspy.teleprompt import BootstrapFewShot, LabeledFewShot, MIPROv2

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

# load your environment variables from .env file
load_dotenv()

# azure-openai model deployment
AZURE_OPENAI_KEY = os.getenv("AZURE_OPENAI_KEY")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_DEPLOYMENT = os.getenv("AZURE_OPENAI_DEPLOYMENT")
AZURE_OPENAI_VERSION = os.getenv("AZURE_OPENAI_VERSION")

## Loading the Dataset

For this tutorial, we are going to use the [HotPotQA dataset](https://hotpotqa.github.io/). The task for the LLM is multi-hop question answering.

In [2]:
dataset = HotPotQA(train_seed=1, train_size=150, eval_seed=2023, dev_size=50, test_size=0)

In [5]:
dataset.train[0]

Example({'question': 'At My Window was released by which American singer-songwriter?', 'answer': 'John Townes Van Zandt'}) (input_keys=None)

### Specifying Inputs

It's crucial to inform DSPy which attributes serve as inputs for our model. We accomplish this using the `.with_inputs()` method.

In [6]:
train_set = [x.with_inputs('question') for x in dataset.train]
dev_set = [x.with_inputs('question') for x in dataset.dev]

In [7]:
train_set[0]

Example({'question': 'At My Window was released by which American singer-songwriter?', 'answer': 'John Townes Van Zandt'}) (input_keys={'question'})

## DSPy setup

DSPy is designed to be compatible with a variety of language models and their respective clients ([click here](https://dspy.ai/learn/programming/language_models) for a complete guide to all supported LLMs). For this tutorial, we will primarily utilize GPT-4 through the Azure client


In [8]:
llm = dspy.AzureOpenAI(
    api_base=AZURE_OPENAI_ENDPOINT,
    api_version=AZURE_OPENAI_VERSION,
    deployment_id=AZURE_OPENAI_DEPLOYMENT,
    api_key=AZURE_OPENAI_KEY,
    # Specify the models parameter
    temperature=0.7
)

retriever = dspy.ColBERTv2(url='http://20.102.90.50:2017/wiki17_abstracts')

dspy.settings.configure(lm=llm, rm=retriever)

## Setting up Signatures and Modules


### Core DSPy Components

[dspy.Signature](https://dspy.ai/learn/programming/signatures/) and [dspy.Module](https://dspy.ai/learn/programming/modules/) are fundamental building blocks for DSPy programs:

- **Signature**: A declarative specification of the input/output behavior of a DSPy module.
- **Module**: A building block for programs that leverage Language Models (LMs).

### Types of DSPy Modules

DSPy offers various module types, each serving different purposes:

1. [dspy.Predict](https://dspy-docs.vercel.app/api/modules/Predict)
   - Basic predictor
   - Maintains the original signature
   - Handles key forms of learning (storing instructions, demonstrations, and LM updates)
   - Most similar to direct LM usage

2. [dspy.ChainOfThought](https://dspy-docs.vercel.app/api/modules/ChainOfThought)
   - Enhances the LM to think step-by-step before producing the final response
   - Modifies the signature to incorporate intermediate reasoning steps
     

3. [Additional Advanced Modules](https://dspy-docs.vercel.app/api/category/modules)
   - DSPy library offers a range of more specialized modules for complex tasks;
   - In this talk, we will explore the [ReAct](https://dspy.ai/deep-dive/modules/react/) module.

### Recommendation for starting

For those new to DSPy, it's advisable to start with `dspy.Predict`. Its simplicity makes it ideal for understanding the basics of DSPy operation. Once you've successfully implemented your program using `dspy.Predict`, you can explore more advanced modules like `dspy.ChainOfThought` to potentially enhance your model's performance.

For an overview of other prompting techniques beyond zero-shot learning, refer to the [Prompting Guide](https://www.promptingguide.ai/techniques). This resource covers various methods that can enhance your DSPy applications as you progress.

In [9]:
class GenerateAnswer(dspy.Signature):
    # [OPTIONAL]: Clarify something about the nature of the task expressed below as a docstring.
    """Answer questions with short factoid answers."""

    # Supply hints on the nature of an input field, expressed as a desc keyword argument for dspy.InputField.
    context = dspy.InputField(desc="may contain relevant facts")
    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words")

In [10]:
class BaselineRAG(dspy.Module):
    def __init__(self, num_passages=3):
        super().__init__()

        self.retrieve = dspy.Retrieve(k=num_passages)
        self.generate_answer = dspy.ChainOfThought(GenerateAnswer)

    def forward(self, question):
        context = self.retrieve(question).passages
        prediction = self.generate_answer(context=context, question=question)
        return dspy.Prediction(context=context, answer=prediction.answer, rationale=prediction.rationale)

In [11]:
baseline = BaselineRAG()

baseline(train_set[1].question)

 		You are using the client AzureOpenAI, which will be removed in DSPy 2.6.
 		Changing the client is straightforward and will let you use new features (Adapters) that improve the consistency of LM outputs, especially when using chat LMs. 

 		Learn more about the changes and how to migrate at
 		https://github.com/stanfordnlp/dspy/blob/main/examples/migration.ipynb


Prediction(
    context=['Candace Kita | Kita\'s first role was as a news anchor in the 1991 movie "Stealth Hunters". Kita\'s first recurring television role was in Fox\'s "Masked Rider", from 1995 to 1996. She appeared as a series regular lead in all 40 episodes. Kita also portrayed a frantic stewardess in a music video directed by Mark Pellington for the British group, Catherine Wheel, titled, "Waydown" in 1995. In 1996, Kita also appeared in the film "Barb Wire" (1996) and guest starred on "The Wayans Bros.". She also guest starred in "Miriam Teitelbaum: Homicide" with "Saturday Night Live" alumni Nora Dunn, "Wall To Wall Records" with Jordan Bridges, "Even Stevens", "Felicity" with Keri Russell, "V.I.P." with Pamela Anderson, "Girlfriends", "The Sweet Spot" with Bill Murray, and "Movies at Our House". She also had recurring roles on the FX spoof, "Son of the Beach" from 2001 to 2002, ABC-Family\'s "Dance Fever" and Oxygen Network\'s "Running with Scissors". Kita also appeared in th

In [11]:
llm.inspect_history(n=1)




Answer questions with short factoid answers.

---

Follow the following format.

Context: may contain relevant facts

Question: ${question}

Reasoning: Let's think step by step in order to ${produce the answer}. We ...

Answer: often between 1 and 5 words

---

Context:
[1] «Candace Kita | Kita's first role was as a news anchor in the 1991 movie "Stealth Hunters". Kita's first recurring television role was in Fox's "Masked Rider", from 1995 to 1996. She appeared as a series regular lead in all 40 episodes. Kita also portrayed a frantic stewardess in a music video directed by Mark Pellington for the British group, Catherine Wheel, titled, "Waydown" in 1995. In 1996, Kita also appeared in the film "Barb Wire" (1996) and guest starred on "The Wayans Bros.". She also guest starred in "Miriam Teitelbaum: Homicide" with "Saturday Night Live" alumni Nora Dunn, "Wall To Wall Records" with Jordan Bridges, "Even Stevens", "Felicity" with Keri Russell, "V.I.P." with Pamela Anderson, "Girlfrien

'\n\n\nAnswer questions with short factoid answers.\n\n---\n\nFollow the following format.\n\nContext: may contain relevant facts\n\nQuestion: ${question}\n\nReasoning: Let\'s think step by step in order to ${produce the answer}. We ...\n\nAnswer: often between 1 and 5 words\n\n---\n\nContext:\n[1] «Candace Kita | Kita\'s first role was as a news anchor in the 1991 movie "Stealth Hunters". Kita\'s first recurring television role was in Fox\'s "Masked Rider", from 1995 to 1996. She appeared as a series regular lead in all 40 episodes. Kita also portrayed a frantic stewardess in a music video directed by Mark Pellington for the British group, Catherine Wheel, titled, "Waydown" in 1995. In 1996, Kita also appeared in the film "Barb Wire" (1996) and guest starred on "The Wayans Bros.". She also guest starred in "Miriam Teitelbaum: Homicide" with "Saturday Night Live" alumni Nora Dunn, "Wall To Wall Records" with Jordan Bridges, "Even Stevens", "Felicity" with Keri Russell, "V.I.P." with Pa

## Metrics & Evaluation

In [12]:
# We have a positive match when:
# (generated answer and true answer match exactly) AND (the retrieved context does actually contain that answer)
def validate_context_and_answer(example, pred, trace=None):
    answer_EM = dspy.evaluate.answer_exact_match(example, pred)
    answer_PM = dspy.evaluate.answer_passage_match(example, pred)
    return answer_EM and answer_PM

In [15]:
evaluate_hotpot = Evaluate(
    devset=dev_set,
    metric=validate_context_and_answer,
    num_threads=8,
    display_progress=True,
    display_table=10,
)

In [16]:
evaluate_hotpot(baseline)

Average Metric: 18.00 / 50 (36.0%): 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 149.65it/s]

2024/12/10 15:45:39 INFO dspy.evaluate.evaluate: Average Metric: 18 / 50 (36.0%)


,question,example_answer,gold_titles,context,pred_answer,rationale,validate_context_and_answer
0,Are both Cangzhou and Qionghai in the Hebei province of China?,no,"{Cangzhou, Qionghai}",['Cangzhou | Cangzhou () is a prefecture-level city in eastern Heb...,"No, only Cangzhou.",determine the location of both cities. We know that Cangzhou is in...,
1,Who conducts the draft in which Marc-Andre Fleury was drafted to t...,National Hockey League,"{2017–18 Pittsburgh Penguins season, 2017 NHL Expansion Draft}",['2017–18 Pittsburgh Penguins season | The 2017–18 Pittsburgh Peng...,NHL,identify the entity responsible for the draft in question. We know...,
2,"The Wings entered a new era, following the retirement of which Can...",Steve Yzerman,"{Steve Yzerman, 2006–07 Detroit Red Wings season}","['Steve Yzerman | Stephen Gregory ""Steve"" Yzerman ( ; born May 9, ...",Steve Yzerman,identify the player mentioned in the context. We see that Steve Yz...,✔️ [True]
3,What river is near the Crichton Collegiate Church?,the River Tyne,"{Crichton Castle, Crichton Collegiate Church}","[""Crichton Collegiate Church | Crichton Collegiate Church is situa...",River Tyne,identify the river associated with Crichton Collegiate Church. The...,✔️ [True]
4,In the 10th Century A.D. Ealhswith had a son called Æthelweard by ...,King Alfred the Great,"{Ealhswith, Æthelweard (son of Alfred)}","[""Æthelweard of East Anglia | Æthelweard (died 854) was a 9th-cent...",Alfred the Great,identify the king who was the father of Æthelweard. Ealhswith is k...,
5,The Newark Airport Exchange is at the northern edge of an airport ...,Port Authority of New York and New Jersey,"{Newark Airport Interchange, Newark Liberty International Airport}",['Newark Airport Interchange | The Newark Airport Interchange is a...,Port Authority of New York and New Jersey,identify the operator of Newark Liberty International Airport. We ...,✔️ [True]
6,Where did an event take place resulting in a win during a domestic...,Bundesliga,"{Claudio Pizarro, 2005–06 FC Bayern Munich season}",['List of Peru international footballers | Peru took part in the i...,Peru,produce the answer. We know that the context mentions important Pe...,
7,Are both Chico Municipal Airport and William R. Fairchild Internat...,no,"{William R. Fairchild International Airport, Chico Municipal Airport}",['William R. Fairchild International Airport | William R. Fairchil...,No,determine the locations of the airports. William R. Fairchild Inte...,✔️ [True]
8,In which Maine county is Fort Pownall located?,"Waldo County, Maine","{Fort Pownall, Stockton Springs, Maine}","[""Fort Pownall | Fort Pownall was a British fortification built du...",Waldo County,identify the location of Fort Pownall. The context specifies that ...,
9,"Which 90s rock band has more recently reformed, Gene or The Afghan...",The Afghan Whigs,"{Gene (band), The Afghan Whigs}",['The Afghan Whigs | The Afghan Whigs are an American rock band fr...,The Afghan Whigs,compare the two bands' timelines. The Afghan Whigs were originally...,✔️ [True]


36.0

## And now... some DSPy magic ✨

### Using a ReAct module

The ReAct framework combines reasoning (thinking) and acting (taking actions) in LLMs.
It alternates between reasoning about the problem and performing actions, like retrieving information or interacting with an environment, to dynamically adapt as new information becomes available.

<center><img src="assets/react-schema.png" width="1000" align="center"><center>

In [13]:
# ReAct is internally already a more complex dspy.Module - we don't need to wrap it into a class
agent = dspy.ReAct("question -> answer", tools=[dspy.Retrieve(k=1)])

In [19]:
question = train_set[1].question
result = agent(question=question)
result.trajectory

{'thought_0': "Candace Kita is an actress known for her roles in various television shows and films. To provide an accurate answer, I need to find specific instances of her guest appearances and the American actors she starred alongside. \n\nNext Thought: I will search for information regarding Candace Kita's guest appearances in TV shows or films to identify the American actors she has worked with.",
 'tool_name_0': 'Search',
 'tool_args_0': '{"query": "Candace Kita guest appearances American actor"}',
 'observation_0': Prediction(
     passages=['Candace Kita | Kita\'s first role was as a news anchor in the 1991 movie "Stealth Hunters". Kita\'s first recurring television role was in Fox\'s "Masked Rider", from 1995 to 1996. She appeared as a series regular lead in all 40 episodes. Kita also portrayed a frantic stewardess in a music video directed by Mark Pellington for the British group, Catherine Wheel, titled, "Waydown" in 1995. In 1996, Kita also appeared in the film "Barb Wire" (

In [20]:
llm.inspect_history(n=1)




Given the fields `question`, produce the fields `answer`.

---

Follow the following format.

Question: ${question}

Trajectory: ${trajectory}

Reasoning: Let's think step by step in order to ${produce the answer}. We ...

Answer: ${answer}

---

Question: which American actor was Candace Kita guest starred with

Trajectory: [[ ## thought_0 ## ]] Candace Kita is an actress known for her roles in various television shows and films. To provide an accurate answer, I need to find specific instances of her guest appearances and the American actors she starred alongside. Next Thought: I will search for information regarding Candace Kita's guest appearances in TV shows or films to identify the American actors she has worked with. [[ ## tool_name_0 ## ]] Search [[ ## tool_args_0 ## ]] {"query": "Candace Kita guest appearances American actor"} [[ ## observation_0 ## ]] Prediction( passages=['Candace Kita | Kita\'s first role was as a news anchor in the 1991 movie "Stealth Hunters". Kita\'s f

'\n\n\nGiven the fields `question`, produce the fields `answer`.\n\n---\n\nFollow the following format.\n\nQuestion: ${question}\n\nTrajectory: ${trajectory}\n\nReasoning: Let\'s think step by step in order to ${produce the answer}. We ...\n\nAnswer: ${answer}\n\n---\n\nQuestion: which American actor was Candace Kita guest starred with\n\nTrajectory: [[ ## thought_0 ## ]] Candace Kita is an actress known for her roles in various television shows and films. To provide an accurate answer, I need to find specific instances of her guest appearances and the American actors she starred alongside. Next Thought: I will search for information regarding Candace Kita\'s guest appearances in TV shows or films to identify the American actors she has worked with. [[ ## tool_name_0 ## ]] Search [[ ## tool_args_0 ## ]] {"query": "Candace Kita guest appearances American actor"} [[ ## observation_0 ## ]] Prediction( passages=[\'Candace Kita | Kita\\\'s first role was as a news anchor in the 1991 movie "

In [19]:
config = dict(num_threads=8, display_progress=True, display_table=5)
evaluate = Evaluate(devset=dev_set, metric=dspy.evaluate.answer_exact_match, **config)

evaluate(agent)


verage Metric: 31.00 / 50 (62.0%): 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:01<00:00, 29.57it/s]

2024/12/10 16:12:43 INFO dspy.evaluate.evaluate: Average Metric: 31 / 50 (62.0%)


,question,example_answer,gold_titles,trajectory,rationale,pred_answer,answer_exact_match
0,Are both Cangzhou and Qionghai in the Hebei province of China?,no,"{Qionghai, Cangzhou}","{'thought_0': 'Cangzhou is located in Hebei province, while Qiongh...",determine whether both Cangzhou and Qionghai are in the Hebei prov...,no,✔️ [True]
1,Who conducts the draft in which Marc-Andre Fleury was drafted to t...,National Hockey League,"{2017–18 Pittsburgh Penguins season, 2017 NHL Expansion Draft}","{'thought_0': ""The question asks about the draft process for Marc-...",determine who conducted the draft for Marc-Andre Fleury's selectio...,National Hockey League,✔️ [True]
2,"The Wings entered a new era, following the retirement of which Can...",Steve Yzerman,"{Steve Yzerman, 2006–07 Detroit Red Wings season}","{'thought_0': ""The Wings' new era likely began after a significant...",identify the retired Canadian professional ice hockey player who w...,Steve Yzerman,✔️ [True]
3,What river is near the Crichton Collegiate Church?,the River Tyne,"{Crichton Castle, Crichton Collegiate Church}",{'thought_0': 'Question: What river is near the Crichton Collegiat...,determine the river near the Crichton Collegiate Church. We start ...,River Tyne,✔️ [True]
4,In the 10th Century A.D. Ealhswith had a son called Æthelweard by ...,King Alfred the Great,"{Ealhswith, Æthelweard (son of Alfred)}","{'thought_0': ""Question: In the 10th Century A.D. Ealhswith had a ...",determine which English king was Ealhswith's husband. We know that...,Alfred the Great,


62.0

### Using Optimizers - Automatic Few-Shot Learning

These optimizers extend the signature by automatically generating and including optimized examples within the prompt sent to the model, implementing few-shot learning.

#### LabeledFewShot

Simply constructs few-shot examples (demos) from provided labeled input and output data points.

In [23]:
optimizer = LabeledFewShot(k=3)
optimized_program = optimizer.compile(agent, trainset=train_set)
optimized_program.save("compiled_programs/react_LabeledFewShot_program.json")

evaluate(optimized_program)

Average Metric: 31.00 / 50 (62.0%): 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:01<00:00, 32.83it/s]

2024/12/10 16:14:34 INFO dspy.evaluate.evaluate: Average Metric: 31 / 50 (62.0%)


,question,example_answer,gold_titles,trajectory,rationale,pred_answer,answer_exact_match
0,Are both Cangzhou and Qionghai in the Hebei province of China?,no,"{Qionghai, Cangzhou}","{'thought_0': 'Cangzhou is located in Hebei province, while Qiongh...",determine whether both Cangzhou and Qionghai are in the Hebei prov...,no,✔️ [True]
1,Who conducts the draft in which Marc-Andre Fleury was drafted to t...,National Hockey League,"{2017–18 Pittsburgh Penguins season, 2017 NHL Expansion Draft}","{'thought_0': ""The question asks about the draft process for Marc-...",determine who conducted the draft for Marc-Andre Fleury's selectio...,National Hockey League,✔️ [True]
2,"The Wings entered a new era, following the retirement of which Can...",Steve Yzerman,"{Steve Yzerman, 2006–07 Detroit Red Wings season}","{'thought_0': ""The Wings' new era likely began after a significant...",identify the retired Canadian professional ice hockey player who w...,Steve Yzerman,✔️ [True]
3,What river is near the Crichton Collegiate Church?,the River Tyne,"{Crichton Castle, Crichton Collegiate Church}",{'thought_0': 'Question: What river is near the Crichton Collegiat...,determine the river near the Crichton Collegiate Church. We start ...,River Tyne,✔️ [True]
4,In the 10th Century A.D. Ealhswith had a son called Æthelweard by ...,King Alfred the Great,"{Ealhswith, Æthelweard (son of Alfred)}","{'thought_0': ""Question: In the 10th Century A.D. Ealhswith had a ...",determine which English king was Ealhswith's husband. We know that...,Alfred the Great,


62.0

#### Loading a "pre-optimized" model

In [22]:
agent.load("compiled_programs/react_LabeledFewShot_program.json")

In [15]:
question = train_set[1].question
result = agent(question=question)
result.trajectory

{'thought_0': "Candace Kita is an actress known for her roles in various television shows and films. To provide an accurate answer, I need to find specific instances of her guest appearances and the American actors she starred alongside. \n\nNext Thought: I will search for information regarding Candace Kita's guest appearances in TV shows or films to identify the American actors she has worked with.",
 'tool_name_0': 'Search',
 'tool_args_0': '{"query": "Candace Kita guest appearances American actor"}',
 'observation_0': Prediction(
     passages=['Candace Kita | Kita\'s first role was as a news anchor in the 1991 movie "Stealth Hunters". Kita\'s first recurring television role was in Fox\'s "Masked Rider", from 1995 to 1996. She appeared as a series regular lead in all 40 episodes. Kita also portrayed a frantic stewardess in a music video directed by Mark Pellington for the British group, Catherine Wheel, titled, "Waydown" in 1995. In 1996, Kita also appeared in the film "Barb Wire" (

In [16]:
llm.inspect_history()




Given the fields `question`, produce the fields `answer`.

---

Question: The Australian food spread developed by Cyril Percy Callister is considered an acquired taste based on what sense?
Answer: taste

Question: Who supervises the subordinate that occupies the majority of Chatan, Japan?
Answer: United States Pacific Command

Question: Are Princess Sumaya University for Technology and Tennessee Technological University from the same country?
Answer: no

---

Follow the following format.

Question: ${question}

Trajectory: ${trajectory}

Reasoning: Let's think step by step in order to ${produce the answer}. We ...

Answer: ${answer}

---

Question: which American actor was Candace Kita guest starred with

Trajectory: [[ ## thought_0 ## ]] Candace Kita is an actress known for her roles in various television shows and films. To provide an accurate answer, I need to find specific instances of her guest appearances and the American actors she starred alongside. Next Thought: I will sear

'\n\n\nGiven the fields `question`, produce the fields `answer`.\n\n---\n\nQuestion: The Australian food spread developed by Cyril Percy Callister is considered an acquired taste based on what sense?\nAnswer: taste\n\nQuestion: Who supervises the subordinate that occupies the majority of Chatan, Japan?\nAnswer: United States Pacific Command\n\nQuestion: Are Princess Sumaya University for Technology and Tennessee Technological University from the same country?\nAnswer: no\n\n---\n\nFollow the following format.\n\nQuestion: ${question}\n\nTrajectory: ${trajectory}\n\nReasoning: Let\'s think step by step in order to ${produce the answer}. We ...\n\nAnswer: ${answer}\n\n---\n\nQuestion: which American actor was Candace Kita guest starred with\n\nTrajectory: [[ ## thought_0 ## ]] Candace Kita is an actress known for her roles in various television shows and films. To provide an accurate answer, I need to find specific instances of her guest appearances and the American actors she starred al

### Using Optimizers - Automatic Instruction Optimization

These optimizers produce optimal instructions for the prompt.

#### MIPROv2

Generates instructions and few-shot examples in each step. The instruction generation is data-aware and demonstration-aware. Uses Bayesian Optimization to effectively search over the space of generation instructions/demonstrations across your modules.

<center><img src="assets/mipro-schema.png" width="400"></center>

In [20]:
optimizer = MIPROv2(
    metric=dspy.evaluate.answer_exact_match,
    auto="light",                      # Can choose between light, medium, and heavy optimization runs
    verbose=True
)
optimized_program = optimizer.compile(
    baseline,
    trainset=train_set,
    max_bootstrapped_demos=0,           # setting this to 0 to optimize only the instruction
    max_labeled_demos=0,                # setting this to 0 to optimize only the instructions
    requires_permission_to_run=True
)

optimized_program.save("compiled_programs/baseline_MIPROv2_program.json")

evaluate(optimized_program)

2024/12/10 16:12:55 INFO dspy.teleprompt.mipro_optimizer_v2: 
RUNNING WITH THE FOLLOWING LIGHT AUTO RUN SETTINGS:
num_trials: 7
minibatch: True
num_candidates: 7
valset size: 100



Projected Language Model (LM) Calls

Based on the parameters you have set, the maximum number of LM calls is projected as follows:

- Prompt Generation: 10 data summarizer calls + 7 * 1 lm calls in program + (2) lm calls in program-aware proposer = 19 prompt model calls
- Program Evaluation: 25 examples in minibatch * 7 batches + 100 examples in val set * 1 full evals = 275 LM Program calls

Estimated Cost Calculation:

Total Cost = (Number of calls to task model * (Avg Input Token Length per Call * Task Model Price per Input Token + Avg Output Token Length per Call * Task Model Price per Output Token) 
            + (Number of program calls * (Avg Input Token Length per Call * Task Prompt Price per Input Token + Avg Output Token Length per Call * Prompt Model Price per Output Token).

For a preliminary estimate of potential costs, we recommend you perform your own calculations based on the task
and prompt models you intend to use. If the projected costs exceed your budget or expectati

2024/12/10 16:12:57 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2024/12/10 16:12:57 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used for informing instruction proposal.

2024/12/10 16:12:57 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=7 sets of demonstrations...


Bootstrapping set 1/7
Bootstrapping set 2/7



17%|█████████████████████████████████████████████                                                                                                                                                                                                                                 | 5/30 [00:00<00:00, 207.97it/s]

...

2024/12/10 16:12:59 INFO dspy.teleprompt.mipro_optimizer_v2: New best full eval score! Score: 41.0
2024/12/10 16:12:59 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [39.0, 41.0]
2024/12/10 16:12:59 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 41.0
2024/12/10 16:12:59 INFO dspy.teleprompt.mipro_optimizer_v2: =======================
2024/12/10 16:12:59 INFO dspy.teleprompt.mipro_optimizer_v2: 

2024/12/10 16:12:59 INFO dspy.teleprompt.mipro_optimizer_v2: Returning best identified program with score 41.0!


Average Metric: 21.00 / 50 (42.0%): 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 244.82it/s]

2024/12/10 16:12:59 INFO dspy.evaluate.evaluate: Average Metric: 21 / 50 (42.0%)


,question,example_answer,gold_titles,context,pred_answer,rationale,answer_exact_match
0,Are both Cangzhou and Qionghai in the Hebei province of China?,no,"{Qionghai, Cangzhou}",['Cangzhou | Cangzhou () is a prefecture-level city in eastern Heb...,"No, only Cangzhou is.",determine the location of both cities. Cangzhou is explicitly ment...,
1,Who conducts the draft in which Marc-Andre Fleury was drafted to t...,National Hockey League,"{2017–18 Pittsburgh Penguins season, 2017 NHL Expansion Draft}",['2017–18 Pittsburgh Penguins season | The 2017–18 Pittsburgh Peng...,National Hockey League,identify the organization responsible for the expansion draft. The...,✔️ [True]
2,"The Wings entered a new era, following the retirement of which Can...",Steve Yzerman,"{Steve Yzerman, 2006–07 Detroit Red Wings season}","['Steve Yzerman | Stephen Gregory ""Steve"" Yzerman ( ; born May 9, ...",Steve Yzerman,identify the player being referred to. The context states that the...,✔️ [True]
3,What river is near the Crichton Collegiate Church?,the River Tyne,"{Crichton Castle, Crichton Collegiate Church}","[""Crichton Collegiate Church | Crichton Collegiate Church is situa...",River Tyne,identify the relevant geographical features. The context mentions ...,✔️ [True]
4,In the 10th Century A.D. Ealhswith had a son called Æthelweard by ...,King Alfred the Great,"{Ealhswith, Æthelweard (son of Alfred)}","[""Æthelweard of East Anglia | Æthelweard (died 854) was a 9th-cent...",Alfred the Great,identify the correct English king associated with Ealhswith and he...,


42.0

In [21]:
llm.inspect_history(n=1)




You are a knowledgeable trivia expert who specializes in providing concise and accurate answers to fact-based questions. Use the provided context to identify the relevant information and respond with a brief answer, ensuring clarity and precision.

---

Follow the following format.

Context: may contain relevant facts

Question: ${question}

Reasoning: Let's think step by step in order to ${produce the answer}. We ...

Answer: often between 1 and 5 words

---

Context:
[1] «Election Law Journal | Election Law Journal is a quarterly peer-reviewed law journal published by Mary Ann Liebert, Inc. covering legal issues related to elections and voting rights. It was established in 2002 with Daniel H. Lowenstein (UCLA School of Law) and Richard L. Hasen (Loyola Law School) as founding editors-in-chief. s of 2010 , the editors are Daniel P. Tokaji (Moritz College of Law) and Paul Gronke (Reed College). The journal is abstracted and indexed in Westlaw and International Political Science Abst

'\n\n\nYou are a knowledgeable trivia expert who specializes in providing concise and accurate answers to fact-based questions. Use the provided context to identify the relevant information and respond with a brief answer, ensuring clarity and precision.\n\n---\n\nFollow the following format.\n\nContext: may contain relevant facts\n\nQuestion: ${question}\n\nReasoning: Let\'s think step by step in order to ${produce the answer}. We ...\n\nAnswer: often between 1 and 5 words\n\n---\n\nContext:\n[1] «Election Law Journal | Election Law Journal is a quarterly peer-reviewed law journal published by Mary Ann Liebert, Inc. covering legal issues related to elections and voting rights. It was established in 2002 with Daniel H. Lowenstein (UCLA School of Law) and Richard L. Hasen (Loyola Law School) as founding editors-in-chief. s of 2010 , the editors are Daniel P. Tokaji (Moritz College of Law) and Paul Gronke (Reed College). The journal is abstracted and indexed in Westlaw and International 